<a href="https://colab.research.google.com/github/AlekhyaGangopadhyay/IITPatna_Internship_LLMs/blob/main/InternVL2_5_8B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ============================================================
#  InternVL2.5-8B  ▸  Finance Harassment/Abuse Video Dataset
#  Full Pipeline : Train (LoRA) → Infer → Accuracy Score
#  Dataset       : iamalekhya/Finance_set  (318 videos, 9 cols)
#  Run on        : Google Colab FREE T4 (16 GB VRAM) ✅
# ============================================================

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# CELL 1 ── Install  (run once → Runtime ▸ Restart session)
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
!pip install transformers>=4.37.2 accelerate bitsandbytes peft
!pip install rouge-score openpyxl scikit-learn opencv-python timm
!pip install flash-attn --no-build-isolation   # optional, speeds up attention on A10/A100
# After install → Runtime ▸ Restart session ▸ then run Cell 2 onwards

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# CELL 2 ── Mount Google Drive
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
from google.colab import drive
drive.mount('/content/drive')

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# CELL 3 ── ⚙️  CONFIG  ← only edit this section
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

MODEL_PATH     = "OpenGVLab/InternVL2_5-8B"    # fits on free T4 in 4-bit

# Dataset
HF_DATASET     = "iamalekhya/Finance_set"
EXCEL_FALLBACK = "/content/drive/MyDrive/IITP_Internship/Finance_set.xlsx"

# Videos in Google Drive (all 5 folders searched automatically)
VIDEO_DIRS = [
    "/content/drive/Shared drives/Finance_video_Anamoly/AG",
    "/content/drive/Shared drives/Finance_video_Anamoly/ag",
    "/content/drive/Shared drives/Finance_video_Anamoly/Alekhya",
    "/content/drive/Shared drives/Finance_video_Anamoly/Pritam",
    "/content/drive/Shared drives/Finance_video_Anamoly/Videos",
]
VIDEO_EXT = ".mp4"

# Training
NUM_FRAMES  = 8       # frames per video  (reduce to 4 if OOM on T4)
IMAGE_SIZE  = 448     # InternVL2.5 native resolution
MAX_SEQ_LEN = 2048
EPOCHS      = 3
BATCH_SIZE  = 1
GRAD_ACCUM  = 8
LORA_RANK   = 16
EVAL_SPLIT  = 0.10
RANDOM_SEED = 42

# Paths
OUTPUT_DIR          = "./internvl25_8b_finance"
ADAPTER_DRIVE       = "/content/drive/MyDrive/IIT_Internship/internvl25_8b_adapter"
RESULTS_EXCEL_DRIVE = "/content/drive/MyDrive/IIT_Internship/internvl25_8b_results.xlsx"

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# CELL 4 ── Imports
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
import os, re, json, cv2, shutil, torch, warnings
import numpy as np
import pandas as pd
import torchvision.transforms as T
from PIL import Image
from torchvision.transforms.functional import InterpolationMode
from transformers import AutoModel, AutoTokenizer, BitsAndBytesConfig
from peft import get_peft_model, LoraConfig, TaskType
from datasets import load_dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from rouge_score import rouge_scorer as rs
from torch.utils.data import Dataset as TorchDataset
from torch.nn.utils.rnn import pad_sequence
from transformers import TrainingArguments, Trainer

warnings.filterwarnings("ignore")
print("✅ Imports OK")

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# CELL 5 ── Image Transform  (official InternVL2.5 pipeline)
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
IMAGENET_MEAN = (0.485, 0.456, 0.406)
IMAGENET_STD  = (0.229, 0.224, 0.225)

def build_transform(size=IMAGE_SIZE):
    return T.Compose([
        T.Lambda(lambda img: img.convert('RGB') if img.mode != 'RGB' else img),
        T.Resize((size, size), interpolation=InterpolationMode.BICUBIC),
        T.ToTensor(),
        T.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD),
    ])

IMG_TRANSFORM = build_transform(IMAGE_SIZE)

def pil_to_pixel_values(pil_images: list) -> torch.Tensor:
    """Stack PIL frames → (N, 3, H, W) float tensor."""
    return torch.stack([IMG_TRANSFORM(img) for img in pil_images])

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# CELL 6 ── Load Dataset
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
print("Loading dataset...")
try:
    ds = load_dataset(HF_DATASET, split="train")
    df = ds.to_pandas()
    print(f"✅ Loaded from HuggingFace: {len(df)} rows")
except Exception as e:
    print(f"⚠️  HF failed ({e}), loading from Excel...")
    df = pd.read_excel(EXCEL_FALLBACK)
    print(f"✅ Loaded from Excel: {len(df)} rows")

df.columns = [c.strip() for c in df.columns]
REQUIRED = ["uni_id","Category","Aspect","Intent","Explanation","Country","Source","Sector"]
missing  = [c for c in REQUIRED if c not in df.columns]
if missing:
    raise ValueError(f"Missing columns: {missing}")

df = df.dropna(subset=REQUIRED).reset_index(drop=True)
print(f"   Rows: {len(df)} | Categories: {df['Category'].unique().tolist()}")

ALL_ASPECTS = sorted(df["Aspect"].unique().tolist())
print(f"   Aspects: {len(ALL_ASPECTS)} unique values")

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# CELL 7 ── Prompt
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
SYSTEM_PROMPT = (
    "You are an expert video analyst specialising in financial harassment, "
    "fraud, abuse, and workplace misconduct. Always respond with valid JSON only."
)

def build_question(n_frames: int) -> str:
    """
    InternVL2.5 multi-image format:
    Each frame needs exactly one <image> tag in the question string.
    """
    frame_tags = "\n".join([f"Frame {i+1}: <image>" for i in range(n_frames)])
    return f"""{frame_tags}

Analyse the video frames above carefully and return a single valid JSON object with EXACTLY these fields:
{{
  "category"   : "Harassment" | "Bullying",
  "aspect"     : one of the known aspect labels listed below,
  "intent"     : 1 (intentional) | 0 (unintentional),
  "explanation": "1-2 sentence description of what is happening and why it is problematic",
  "country"    : "country name or Unknown",
  "source"     : "YouTube | News | Training | Documentary | Other",
  "sector"     : "Finance | Healthcare | Education | Retail | Tech | Government | General"
}}

Known aspect labels (pick the closest match):
{json.dumps(ALL_ASPECTS, indent=2)}

Rules:
- Output ONLY the JSON. No preamble, no markdown fences, no extra text.
- Use double quotes for all strings.
- intent must be an integer (1 or 0), not a string.

Observe carefully:
• Who is involved and what is happening
• The type and form of misconduct
• Whether the act appears deliberate
• Country and industry context clues"""

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# CELL 8 ── Helpers
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
def find_video(uni_id: str) -> str:
    filename = f"{uni_id}{VIDEO_EXT}"
    for folder in VIDEO_DIRS:
        path = os.path.join(folder, filename)
        if os.path.exists(path):
            return path
        if os.path.isdir(folder):
            for f in os.listdir(folder):
                if f.lower() == filename.lower():
                    return os.path.join(folder, f)
    return ""


def extract_frames(video_path: str, n: int = NUM_FRAMES) -> list:
    if not video_path or not os.path.exists(video_path):
        return []
    cap   = cv2.VideoCapture(video_path)
    total = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    if total == 0:
        cap.release()
        return []
    frames = []
    for idx in np.linspace(0, total - 1, n, dtype=int):
        cap.set(cv2.CAP_PROP_POS_FRAMES, idx)
        ok, frame = cap.read()
        if ok:
            frames.append(Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)))
    cap.release()
    return frames


def row_to_json(row) -> str:
    return json.dumps({
        "category"   : str(row["Category"]).strip(),
        "aspect"     : str(row["Aspect"]).strip(),
        "intent"     : int(row["Intent"]),
        "explanation": str(row["Explanation"]).strip(),
        "country"    : str(row["Country"]).strip(),
        "source"     : str(row["Source"]).strip(),
        "sector"     : str(row["Sector"]).strip(),
    }, indent=2)


def parse_json_response(text: str) -> dict:
    try:
        m = re.search(r'\{.*\}', text, re.DOTALL)
        if m:
            return json.loads(m.group())
    except Exception:
        pass
    return {}

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# CELL 9 ── Load Model + Tokenizer
#
#  8B model fits on T4 (16 GB) using 4-bit quantisation.
#  NOTE: Unlike 38B, InternVL2.5-8B's ViT is small enough
#        that 4-bit is safe — no accuracy degradation.
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
print(f"\nLoading {MODEL_PATH} in 4-bit on T4...")

tokenizer = AutoTokenizer.from_pretrained(
    MODEL_PATH,
    trust_remote_code = True,
    use_fast          = False,
)

# 4-bit NF4 quantisation — fits 8B model in ~8 GB VRAM leaving room for training
quant_config = BitsAndBytesConfig(
    load_in_4bit               = True,
    bnb_4bit_quant_type        = "nf4",
    bnb_4bit_use_double_quant  = True,
    bnb_4bit_compute_dtype     = torch.bfloat16,
)

model = AutoModel.from_pretrained(
    MODEL_PATH,
    quantization_config = quant_config,
    low_cpu_mem_usage   = True,
    trust_remote_code   = True,
    use_flash_attn      = False,   # flash-attn not compatible with 4-bit on T4
).eval()

print("✅ Model loaded")

# Apply LoRA to language_model backbone only (vision encoder stays frozen)
lora_config = LoraConfig(
    task_type      = TaskType.CAUSAL_LM,
    r              = LORA_RANK,
    lora_alpha     = LORA_RANK * 2,
    lora_dropout   = 0.05,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj"],
    bias           = "none",
)
model.language_model = get_peft_model(model.language_model, lora_config)
model.language_model.print_trainable_parameters()
print("✅ LoRA applied")

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# CELL 10 ── Build Samples
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
train_df, eval_df = train_test_split(df, test_size=EVAL_SPLIT, random_state=RANDOM_SEED)
train_df = train_df.reset_index(drop=True)
eval_df  = eval_df.reset_index(drop=True)
print(f"\nSplit → Train: {len(train_df)}  |  Eval: {len(eval_df)}")


def build_samples(dataframe, split_name="train"):
    samples, skipped = [], 0
    for i, row in dataframe.iterrows():
        uni_id     = str(row["uni_id"]).strip()
        video_path = find_video(uni_id)
        frames     = extract_frames(video_path)

        if not frames:
            print(f"  ⚠️  [{split_name}] Skipping {uni_id} — video not found")
            skipped += 1
            continue

        samples.append({
            "pil_images" : frames,
            "question"   : build_question(len(frames)),
            "answer"     : row_to_json(row),
            "uni_id"     : uni_id,
            "target_json": row_to_json(row),
        })

        if (i + 1) % 30 == 0:
            print(f"  [{split_name}] {i+1}/{len(dataframe)}  skipped={skipped}")

    print(f"  ✅ [{split_name}] ready={len(samples)}  skipped={skipped}")
    return samples


print("\n📦 Building training samples...")
train_samples = build_samples(train_df, "train")

print("\n📦 Building eval samples...")
eval_samples  = build_samples(eval_df,  "eval")

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# CELL 11 ── PyTorch Dataset + Collator
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
IMG_CONTEXT_TOKEN = '<IMG_CONTEXT>'
IMG_TOKEN         = '<image>'

class InternVL25Dataset(TorchDataset):
    """
    Tokenises samples using InternVL2.5 chat template:
      <|im_start|>system\\n{system}<|im_end|>
      <|im_start|>user\\n{question with <IMG_CONTEXT> expanded}<|im_end|>
      <|im_start|>assistant\\n{answer}<|im_end|>

    Each <image> placeholder is expanded to num_image_token copies
    of <IMG_CONTEXT> before tokenisation — this is mandatory for
    InternVL2.5 to correctly align visual tokens with text tokens.
    """
    def __init__(self, samples, tokenizer, model, max_length=2048):
        self.samples         = samples
        self.tokenizer       = tokenizer
        self.max_length      = max_length
        self.num_image_token = model.num_image_token  # 256 for ViT-448

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        s          = self.samples[idx]
        pil_images = s["pil_images"]
        question   = s["question"]
        answer     = s["answer"]

        # Convert PIL frames → pixel_values
        pixel_values = pil_to_pixel_values(pil_images)  # (N, 3, 448, 448)

        # Expand each <image> → 256x <IMG_CONTEXT>
        expanded_q = question.replace(
            IMG_TOKEN,
            IMG_CONTEXT_TOKEN * self.num_image_token
        )

        # Full conversation string in InternVL2.5 template format
        full_text = (
            f"<|im_start|>system\n{SYSTEM_PROMPT}<|im_end|>"
            f"<|im_start|>user\n{expanded_q}<|im_end|>"
            f"<|im_start|>assistant\n{answer}<|im_end|>"
        )

        encoding = self.tokenizer(
            full_text,
            return_tensors = "pt",
            max_length     = self.max_length,
            truncation     = True,
            padding        = False,
        )
        input_ids = encoding["input_ids"].squeeze(0)

        # Build labels — mask user + system portion, only train on assistant answer
        labels = input_ids.clone()

        # Find where assistant response starts
        assistant_header = self.tokenizer.encode(
            "<|im_start|>assistant\n", add_special_tokens=False
        )
        header_len = len(assistant_header)
        mask_until = 0
        for pos in range(len(input_ids) - header_len):
            if input_ids[pos : pos + header_len].tolist() == assistant_header:
                mask_until = pos + header_len   # train from answer onwards only

        labels[:mask_until] = -100

        # Mask padding tokens
        pad_id = self.tokenizer.pad_token_id or self.tokenizer.eos_token_id
        labels[labels == pad_id] = -100

        return {
            "input_ids"     : input_ids,
            "attention_mask": torch.ones_like(input_ids),
            "labels"        : labels,
            "pixel_values"  : pixel_values,   # (N_frames, 3, 448, 448)
        }


def collate_fn(batch):
    pad_id = tokenizer.pad_token_id or tokenizer.eos_token_id

    input_ids      = pad_sequence([b["input_ids"]      for b in batch], batch_first=True, padding_value=pad_id)
    attention_mask = pad_sequence([b["attention_mask"] for b in batch], batch_first=True, padding_value=0)
    labels         = pad_sequence([b["labels"]         for b in batch], batch_first=True, padding_value=-100)

    # Concatenate all frames across the batch
    pixel_values = torch.cat([b["pixel_values"] for b in batch], dim=0)

    return {
        "input_ids"     : input_ids,
        "attention_mask": attention_mask,
        "labels"        : labels,
        "pixel_values"  : pixel_values,
    }


train_torch_ds = InternVL25Dataset(train_samples, tokenizer, model, max_length=MAX_SEQ_LEN)
print(f"✅ PyTorch dataset ready: {len(train_torch_ds)} training samples")

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# CELL 12 ── Train
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
model.train()
model.language_model.enable_input_require_grads()   # needed for LoRA + gradient checkpointing

training_args = TrainingArguments(
    output_dir                    = OUTPUT_DIR,
    per_device_train_batch_size   = BATCH_SIZE,
    gradient_accumulation_steps   = GRAD_ACCUM,
    num_train_epochs              = EPOCHS,
    learning_rate                 = 2e-4,
    bf16                          = True,
    logging_steps                 = 5,
    save_steps                    = 100,
    save_total_limit              = 2,
    warmup_ratio                  = 0.03,
    lr_scheduler_type             = "cosine",
    optim                         = "paged_adamw_8bit",   # saves VRAM on T4
    seed                          = RANDOM_SEED,
    remove_unused_columns         = False,
    dataloader_pin_memory         = False,
    gradient_checkpointing        = True,
    gradient_checkpointing_kwargs = {"use_reentrant": False},
    report_to                     = "none",
)

trainer = Trainer(
    model         = model,
    args          = training_args,
    train_dataset = train_torch_ds,
    data_collator = collate_fn,
)

print("\n🚀 Training started...")
trainer.train()
print("✅ Training complete!")

# Save LoRA adapter
model.language_model.save_pretrained("internvl25_8b_lora_adapter")
tokenizer.save_pretrained("internvl25_8b_lora_adapter")
os.makedirs(ADAPTER_DRIVE, exist_ok=True)
shutil.copytree("internvl25_8b_lora_adapter", ADAPTER_DRIVE, dirs_exist_ok=True)
print(f"✅ Adapter saved → {ADAPTER_DRIVE}")

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# CELL 13 ── Inference Function
#
#  model.chat() is InternVL's official inference API.
#  It handles <image> expansion + generation internally.
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
model.eval()

GENERATION_CONFIG = dict(
    max_new_tokens = 512,
    do_sample      = False,
    temperature    = 0.1,
    num_beams      = 1,
)

def predict(pil_images: list) -> dict:
    """
    Run inference on a list of PIL frames using model.chat().
    pixel_values must be cast to bfloat16 to match model dtype.
    """
    pixel_values = pil_to_pixel_values(pil_images).to(
        dtype  = torch.bfloat16,
        device = model.device,
    )
    question = build_question(len(pil_images))

    with torch.no_grad():
        response = model.chat(
            tokenizer         = tokenizer,
            pixel_values      = pixel_values,
            question          = question,
            generation_config = GENERATION_CONFIG,
        )

    return parse_json_response(response)

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# CELL 14 ── Run Inference on ALL samples (train + eval)
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
print("\n" + "="*65)
print("  🔍 Running inference on ALL samples (train + eval)")
print("="*65)

all_samples = train_samples + eval_samples
all_results = []

for i, s in enumerate(all_samples):
    pred = predict(s["pil_images"])
    tgt  = json.loads(s["target_json"])

    all_results.append({
        "uni_id"          : s["uni_id"],
        "split"           : "train" if i < len(train_samples) else "eval",
        "true_category"   : tgt.get("category",    ""),
        "true_aspect"     : tgt.get("aspect",       ""),
        "true_intent"     : str(tgt.get("intent",   "")),
        "true_explanation": tgt.get("explanation",  ""),
        "true_country"    : tgt.get("country",      ""),
        "true_source"     : tgt.get("source",       ""),
        "true_sector"     : tgt.get("sector",       ""),
        "pred_category"   : str(pred.get("category",    "")).strip(),
        "pred_aspect"     : str(pred.get("aspect",       "")).strip(),
        "pred_intent"     : str(pred.get("intent",       "")).strip(),
        "pred_explanation": str(pred.get("explanation",  "")).strip(),
        "pred_country"    : str(pred.get("country",      "")).strip(),
        "pred_source"     : str(pred.get("source",       "")).strip(),
        "pred_sector"     : str(pred.get("sector",       "")).strip(),
    })

    if (i + 1) % 20 == 0:
        print(f"  Inferred {i+1}/{len(all_samples)}")

print(f"✅ Inference done for {len(all_results)} samples")

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# CELL 15 ── Accuracy Scoring
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
scorer = rs.RougeScorer(["rougeL"], use_stemmer=True)

def score_split(rows, label="ALL"):
    if not rows:
        return {}

    def acc(t, p):
        return accuracy_score(
            [x.lower().strip() for x in t],
            [x.lower().strip() for x in p]
        )

    rouge_scores = [
        scorer.score(r["true_explanation"], r["pred_explanation"])["rougeL"].fmeasure
        for r in rows
    ]
    avg_rouge = float(np.mean(rouge_scores))

    cat_acc = acc([r["true_category"] for r in rows], [r["pred_category"] for r in rows])
    asp_acc = acc([r["true_aspect"]   for r in rows], [r["pred_aspect"]   for r in rows])
    int_acc = acc([r["true_intent"]   for r in rows], [r["pred_intent"]   for r in rows])
    cty_acc = acc([r["true_country"]  for r in rows], [r["pred_country"]  for r in rows])
    sec_acc = acc([r["true_sector"]   for r in rows], [r["pred_sector"]   for r in rows])
    src_acc = acc([r["true_source"]   for r in rows], [r["pred_source"]   for r in rows])

    overall = (cat_acc*0.25 + asp_acc*0.25 + int_acc*0.15 +
               avg_rouge*0.20 + sec_acc*0.10 + cty_acc*0.03 + src_acc*0.02)

    return {
        "split"             : label,
        "n_samples"         : len(rows),
        "category_acc"      : round(cat_acc   * 100, 2),
        "aspect_acc"        : round(asp_acc   * 100, 2),
        "intent_acc"        : round(int_acc   * 100, 2),
        "country_acc"       : round(cty_acc   * 100, 2),
        "source_acc"        : round(src_acc   * 100, 2),
        "sector_acc"        : round(sec_acc   * 100, 2),
        "explanation_rougeL": round(avg_rouge * 100, 2),
        "OVERALL_SCORE_%"   : round(overall   * 100, 2),
    }


train_rows   = [r for r in all_results if r["split"] == "train"]
eval_rows    = [r for r in all_results if r["split"] == "eval"]
train_scores = score_split(train_rows,  "TRAIN")
eval_scores  = score_split(eval_rows,   "EVAL")
all_scores   = score_split(all_results, "ALL")

def print_scores(s: dict):
    print(f"\n  ── {s['split']} SET  ({s['n_samples']} samples) ──")
    print(f"   Category    Accuracy  : {s['category_acc']:>7.2f} %   (weight 25%)")
    print(f"   Aspect      Accuracy  : {s['aspect_acc']:>7.2f} %   (weight 25%)")
    print(f"   Intent      Accuracy  : {s['intent_acc']:>7.2f} %   (weight 15%)")
    print(f"   Explanation ROUGE-L   : {s['explanation_rougeL']:>7.2f} %   (weight 20%)")
    print(f"   Sector      Accuracy  : {s['sector_acc']:>7.2f} %   (weight 10%)")
    print(f"   Country     Accuracy  : {s['country_acc']:>7.2f} %   (weight  3%)")
    print(f"   Source      Accuracy  : {s['source_acc']:>7.2f} %   (weight  2%)")
    print(f"   {'─'*42}")
    print(f"   ⭐ OVERALL SCORE      : {s['OVERALL_SCORE_%']:>7.2f} %")

print("\n" + "="*65)
print("              📊 ACCURACY REPORT")
print("="*65)
print_scores(train_scores)
print_scores(eval_scores)
print_scores(all_scores)
print("="*65)

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# CELL 16 ── Save Results to Excel (7 sheets)
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
results_df = pd.DataFrame(all_results)
summary_df = pd.DataFrame([train_scores, eval_scores, all_scores])

os.makedirs(os.path.dirname(RESULTS_EXCEL_DRIVE), exist_ok=True)

with pd.ExcelWriter(RESULTS_EXCEL_DRIVE, engine="openpyxl") as writer:
    summary_df.to_excel(writer, sheet_name="Accuracy Summary", index=False)
    results_df.to_excel(writer, sheet_name="Per Sample",       index=False)

    for field in ["category", "aspect", "intent", "sector", "country", "source"]:
        breakdown = results_df[[
            "uni_id", "split",
            f"true_{field}", f"pred_{field}"
        ]].copy()
        breakdown["correct"] = (
            breakdown[f"true_{field}"].str.lower().str.strip() ==
            breakdown[f"pred_{field}"].str.lower().str.strip()
        )
        breakdown.to_excel(writer, sheet_name=f"{field.capitalize()} Detail", index=False)

print(f"\n✅ Results saved → {RESULTS_EXCEL_DRIVE}")
print("\n🎉 ALL DONE!")